In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%cd '/content/drive/MyDrive/KDT/비정형텍스트분석/KDT_1차프로젝트/최종 제출 코드/최종 뉴스 분류 모델'

/content/drive/.shortcut-targets-by-id/1pvIqpy8sVkkUr2OgpRrXK2RrbzxuRmZ0/KDT_1차프로젝트/최종 제출 코드/최종 뉴스 분류 모델


In [ ]:
!pip install transformers

In [ ]:
!curl -s https://raw.githubusercontent.com/teddylee777/machine-learning/master/99-Misc/01-Colab/mecab-colab.sh | bash

In [ ]:
!pip install konlpy

In [ ]:
! pip install python-mecab-ko

In [ ]:
!pip install -q -U sentence-transformers

In [ ]:
import numpy as np
from tqdm import tqdm
import tensorflow as tf
import pandas as pd
from transformers import BertTokenizer, TFBertForSequenceClassification
import nltk
import csv
from konlpy.tag import Hannanum, Okt
import re
from mecab import MeCab
from sklearn.ensemble import RandomForestClassifier
from konlpy.tag import Okt
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import joblib
from sklearn.ensemble import AdaBoostClassifier
from sklearn.metrics import accuracy_score
from sentence_transformers import SentenceTransformer
import matplotlib.pyplot as plt
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Embedding
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.models import load_model
import pickle
import warnings
warnings.filterwarnings('ignore')
tf.random.set_seed(11)

In [ ]:
def news_predict(news, model):


  if model == 'rf':

    print(f'모델: rf')

    # 텍스트 토큰화
    okt = Okt()
    news_title = re.sub(r'[^ㄱ-ㅎㅏ-ㅣ가-힣 ]', '', news)
    news_title = okt.morphs(news_title, norm=True, stem=True)

    print(f'분류 할 제목: {news}')
    print(f'토큰화 된 제목: {news_title}')


    # 모델 경로
    classifier_path = '/content/drive/MyDrive/KDT/비정형텍스트분석/KDT_1차프로젝트/최종 제출 코드/최종 뉴스 분류 모델/classifier_model/b_okt_keras_rf_saved_model1.pkl'
    prepro_path = '/content/drive/MyDrive/KDT/비정형텍스트분석/KDT_1차프로젝트/최종 제출 코드/최종 뉴스 분류 모델/tokenizer/b_okt_keras.pickle'

    # 학습된 임베딩 모델 불러오기
    with open(prepro_path, 'rb') as f:
      a_prepro_model = pickle.load(f)

    # 불러온 임베딩 모델로 처리
    a_prepro_model.fit_on_texts(news_title)
    encoded = a_prepro_model.texts_to_sequences([news_title])
    final_sentence = pad_sequences(encoded, maxlen=9)
    print(f'임베딩 된 제목: {final_sentence}')

    # 학습된 분류 모델 불러오기
    classifier_model = joblib.load(classifier_path)

    # 불러온 분류모델에 임베딩된 뉴스 제목 삽입
    result = float(classifier_model.predict(final_sentence))

    if(result == 1):
      print("상향")
    else:
      print("하향")

#------------------------------------------------------------------------------

  elif model == 'nb':

    print(f'모델: nb')

    # 텍스트 토큰화: 없음
    print(f'분류 할 제목: {news}')
    print(f'토큰화 된 제목: 토큰화 별도로 없음')

    # 모델 경로
    classifier_path = '/content/drive/MyDrive/KDT/비정형텍스트분석/KDT_1차프로젝트/최종 제출 코드/최종 뉴스 분류 모델/classifier_model/nb_sbert.pkl'

    # 학습된 임베딩 모델 불러오기
    model = SentenceTransformer('jhgan/ko-sroberta-multitask')

    # 불러온 임베딩 모델로 처리
    final_sentence = model.encode(news).reshape(1,-1)
    print(f'임베딩 된 제목: 길어서 생략')

    # 학습된 분류 모델 불러오기
    loaded_model = joblib.load(classifier_path)

    # 불러온 분류모델에 임베딩된 뉴스 제목 삽입
    result = float(loaded_model.predict(final_sentence))

    if(result == 1):
      print("상향")
    else:
      print("하향")

#------------------------------------------------------------------------------

  elif model == 'adaboost':

    print(f'모델: adaboost')

    #### ada + b_mecab_keras

    # 텍스트 토큰화
    mecab = MeCab()
    news_title = re.sub(r'[^ㄱ-ㅎㅏ-ㅣ가-힣 ]', '', news)
    news_title = mecab.morphs(news_title)

    print(f'분류 할 제목: {news}')
    print(f'토큰화 된 제목: {news_title}')


    # 모델 경로
    classifier_path = '/content/drive/MyDrive/KDT/비정형텍스트분석/KDT_1차프로젝트/최종 제출 코드/최종 뉴스 분류 모델/classifier_model/ada_b_mecab_keras.pkl'
    prepro_path = '/content/drive/MyDrive/KDT/비정형텍스트분석/KDT_1차프로젝트/최종 제출 코드/최종 뉴스 분류 모델/tokenizer/b_mecab_keras.pickle'

    # 학습된 임베딩 모델 불러오기
    with open(prepro_path, 'rb') as f:
      b_prepro_model = pickle.load(f)

    # 불러온 임베딩 모델로 처리
    b_prepro_model.fit_on_texts(news_title)
    encoded = b_prepro_model.texts_to_sequences([news_title])
    final_sentence = pad_sequences(encoded, maxlen=9)
    print(f'임베딩 된 제목: {final_sentence}')

    # 학습된 분류 모델 불러오기
    classifier_model = joblib.load(classifier_path)

    # 불러온 분류모델에 임베딩된 뉴스 제목 삽입
    result = float(classifier_model.predict(final_sentence))

    if(result == 1):
      print("상향")
    else:
      print("하향")

#------------------------------------------------------------------------------

  elif model == 'lstm':

    print(f'모델: lstm')

    #### lstm + a_mecab_keras

    # 텍스트 토큰화
    mecab = MeCab()
    news_title = re.sub(r'[^ㄱ-ㅎㅏ-ㅣ가-힣 ]', '', news)
    news_title = mecab.morphs(news_title)

    print(f'분류 할 제목: {news}')
    print(f'토큰화 된 제목: {news_title}')

    # 모델 경로
    classifier_path = '/content/drive/MyDrive/KDT/비정형텍스트분석/KDT_1차프로젝트/최종 제출 코드/최종 뉴스 분류 모델/classifier_model/a_mecab_keras_model.h5'
    prepro_path = '/content/drive/MyDrive/KDT/비정형텍스트분석/KDT_1차프로젝트/최종 제출 코드/최종 뉴스 분류 모델/tokenizer/a_mecab_keras.pickle'

    # 학습된 임베딩 모델 불러오기
    with open(prepro_path, 'rb') as f:
      c_prepro_model = pickle.load(f)

    # 불러온 임베딩 모델로 처리
    c_prepro_model.fit_on_texts(news_title)
    encoded = c_prepro_model.texts_to_sequences([news_title])
    final_sentence = pad_sequences(encoded, maxlen=9)
    print(f'임베딩 된 제목: {final_sentence}')

    # 학습된 분류 모델 불러오기
    classifier_model = tf.keras.models.load_model(classifier_path)

    # 불러온 분류모델에 임베딩된 뉴스 제목 삽입
    result = float(classifier_model.predict(final_sentence))

    if(result == 1):
      print("상향")
    else:
      print("하향")

#------------------------------------------------------------------------------

  elif model == 'bert':

    print(f'모델: bert')

    #### bert + hannanum

    # 텍스트 토큰화
    # '제목' 형태소 분석

    hannanum = Hannanum()
    morphs_hannanum = []

    # for news_title in data[title]: #전체 input시 반복문 사용
    morphs_hannanum.append(hannanum.morphs(news))


    ###################################################
    # 불용어 리스트 생성
    f = open('stop_words.txt', encoding = 'utf8')
    reader = csv.reader(f)
    stopwords = []
    for i in reader:
        stopwords.append(i)
    f.close()

    # 2차원으로 반환되므로 1차원리스트로 변환
    stopword_list = np.concatenate(stopwords).tolist()
    ##################################################
    # 불용어 함수
    def remove_stopwords(morphs):
        clean_word = []
        clean_list = []
        for word in morphs:
            #print('불용어 제거 문장\n',word)
            for token in word:
                if token not in stopword_list:
                    clean_word.append(token)

            clean_list.append(clean_word)
            clean_word = []
        return clean_list
    # 함수 사용 -> 불용어 제거 hannanum
    clean_hannanum = remove_stopwords(morphs_hannanum)
    #################################################
    # 문장화 함수
    def make_sentence(clean_list):
        sentence_list = []
        for row in clean_list:
            sentence_list.append(' '.join(row))
        return sentence_list
    # 함수 사용 -> 문장화
    news_title = make_sentence(clean_hannanum)

    print(f'분류 할 제목: {news}')
    print(f'토큰화 된 제목: {news_title}')


    # 모델 불러오기
    save_path = '/content/drive/MyDrive/KDT/비정형텍스트분석/KDT_1차프로젝트/최종 제출 코드/최종 뉴스 분류 모델/classifier_model/ c_hannanum_bert_model'

    # 학습된 모델 불러오기
    loaded_model = TFBertForSequenceClassification.from_pretrained(save_path)

    # input news_title 전처리 (2) bert형식으로 변환 함수
    max_len = 9
    klue_tokenizer = BertTokenizer.from_pretrained('klue/bert-base')
    def klue_bert_tokenizer(sentence, max_len):
      encoded_dict = klue_tokenizer.encode_plus(
          text=sentence,
          add_special_tokens=True,
          max_length=max_len,
          pad_to_max_length=True,
          return_attention_mask=True
      )
      input_id = encoded_dict['input_ids']
      attention_mask = encoded_dict['attention_mask']
      token_type_id = encoded_dict['token_type_ids']
      return input_id, attention_mask, token_type_id

    print(f'임베딩 된 제목: 없음 ')
    # input news_title 전처리 (3) bert형식으로 변환 후: final_sentence -> input_id, attention_mask, token_type_id
    input_id, attention_mask, token_type_id = klue_bert_tokenizer(news_title, max_len)

    input_id = np.array(input_id, dtype=int).reshape(1, -1)
    attention_mask = np.array(attention_mask, dtype=int).reshape(1, -1)
    token_type_id = np.array(token_type_id, dtype=int).reshape(1, -1)

    # input news_title 전처리 (4) 결과 도출
    predictions = loaded_model.predict([input_id, attention_mask, token_type_id])

    result = float(np.argmax(predictions.logits[0]))

    if(result == 1):
      print("상향")
    else:
      print("하향")

In [ ]:
aaa = '코스피, 2600선 안착...기관 매수 유입에 상승세 지속'

In [ ]:
bbb = '삼성전자·SK하이닉스 목표주가 줄상향'

In [ ]:
bbbb = '코스피, 2600선 안착...기관 매수 유입에 상승세 지속'

In [ ]:
민준 = '풍부할수록 좋다…TV 점점 얇아지더니 인기 폭발한 가전'

In [ ]:
하은 = '뚝뚝 떨어진 D램 가격… 삼성전자·SK하이닉스, 2분기도 어둡다'

In [ ]:
news_predict(민준, model='rf')

모델: rf
분류 할 제목: 풍부할수록 좋다…TV 점점 얇아지더니 인기 폭발한 가전
토큰화 된 제목: ['풍부하다', '좋다', '점점', '얇다', '인기', '폭발', '한', '가전']
임베딩 된 제목: [[  0 493  39 162 494 495 496 497 498]]
하향


In [ ]:
news_predict(민준, model='nb')

모델: nb
분류 할 제목: 풍부할수록 좋다…TV 점점 얇아지더니 인기 폭발한 가전
토큰화 된 제목: 토큰화 별도로 없음
임베딩 된 제목: 길어서 생략
상향


In [ ]:
news_predict(민준, model='adaboost')

모델: adaboost
분류 할 제목: 풍부할수록 좋다…TV 점점 얇아지더니 인기 폭발한 가전
토큰화 된 제목: ['풍부', '할수록', '좋', '다', '점점', '얇', '아', '지', '더니', '인기', '폭발', '한', '가전']
임베딩 된 제목: [[147 443 444 445 446 447 448 449 450]]
하향


In [ ]:
news_predict(민준, model='lstm')

모델: lstm
분류 할 제목: 풍부할수록 좋다…TV 점점 얇아지더니 인기 폭발한 가전
토큰화 된 제목: ['풍부', '할수록', '좋', '다', '점점', '얇', '아', '지', '더니', '인기', '폭발', '한', '가전']
임베딩 된 제목: [[126 374 375 376 377 378 379 380 381]]
1/1 [==============================] - 1s 502ms/step
하향


In [ ]:
news_predict(민준, model='bert')

모델: bert
분류 할 제목: 풍부할수록 좋다…TV 점점 얇아지더니 인기 폭발한 가전
토큰화 된 제목: ['풍부 좋다…TV 점점 얇 지 더니 인기 폭발 가전']


Some layers from the model checkpoint at /content/drive/MyDrive/KDT/비정형텍스트분석/KDT_1차프로젝트/최종 제출 코드/최종 뉴스 분류 모델/classifier_model/ c_hannanum_bert_model were not used when initializing TFBertForSequenceClassification: ['dropout_417']
- This IS expected if you are initializing TFBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertForSequenceClassification were initialized from the model checkpoint at /content/drive/MyDrive/KDT/비정형텍스트분석/KDT_1차프로젝트/최종 제출 코드/최종 뉴스 분류 모델/classifier_model/ c_hannanum_bert

임베딩 된 제목: 없음 
1/1 [==============================] - 5s 5s/step
하향
